<a href="https://colab.research.google.com/github/LaTarn14/Relation-Extraction/blob/main/Relation_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install thai-nner**

In [ ]:
import re
!pip install thai-nner==0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**convert model**

In [ ]:
!git clone https://github.com/vistec-AI/Thai-NNER.git

Cloning into 'Thai-NNER'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 254 (delta 6), reused 5 (delta 2), pack-reused 239
Receiving objects: 100% (254/254), 2.73 MiB | 15.62 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [ ]:
cd Thai-NNER

/content/Thai-NNER


In [ ]:
# !gdown 1OAnDe5ZxbS-09S5zT5jFucwdknk1iAu1

Downloading...
From: https://drive.google.com/uc?id=1OAnDe5ZxbS-09S5zT5jFucwdknk1iAu1
To: /content/Thai-NNER/checkpoint.pth
100% 1.29G/1.29G [00:05<00:00, 225MB/s]


In [ ]:
!gdown 1tHkx4ZsQNlGqPE8c7TCSHnpKPNaGtHEK

In [ ]:
!python convert_model2use.py -i checkpoint.pth -o model.pth

In [ ]:
!cp model.pth ../model.pth

In [ ]:
cd ..

/content


In [ ]:
!rm -rf Thai-NNER

**Usage**

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0" # for non-gpu: os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [ ]:
from thai_nner import NNER

In [ ]:
nner = NNER(path_model="model.pth",max_sent_length=512)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Experiment

In [ ]:
def get_ner(sent):
    try:
      return nner.get_tag(sent, show_result = False)
    except:
      try:
        return nner.get_tag(sent[:1250], show_result = False)
      except:
        try: 
          return nner.get_tag(sent[:1200], show_result = False)
        except:
          try:
            return nner.get_tag(sent[:1150], show_result = False)
          except:
            try:
              return nner.get_tag(sent[:1100], show_result = False)
            except:
              try:
                return nner.get_tag(sent[:1050], show_result = False)
              except:
                try:
                  return nner.get_tag(sent[:1000], show_result = False)
                except:
                  return None

function for extract **entities (tuples)** from **pattern (regex)**

In [ ]:
def find_tuples(ner, keyword, n1, n2):
  pat_index = 0
  if keyword in ner[0]:
    pat_index = ner[0].index(keyword)
  nearest_left = -1000000
  nearest_left_entity = ''
  nearest_right_entity = ''
  if len(ner) != 1 and pat_index != 0:
   for i in range(len(ner[1])):
    if type(n1) == list:
     for d in n1:
      if ner[1][i]['entity_type'] == d:
        different_left = ner[1][i]['span'][0] - pat_index
        if 0 > different_left and different_left > nearest_left:
          nearest_left = different_left
          nearest_left_entity = ''.join(ner[1][i]['text'])
    else:
     if ner[1][i]['entity_type'] == n1:
       different_left = ner[1][i]['span'][0] - pat_index
       if 0 > different_left and different_left > nearest_left:
         nearest_left = different_left
         nearest_left_entity = ''.join(ner[1][i]['text'])
    if type(n2) == list:
     for f in n2:
      if re.search(f,ner[1][i]['entity_type']) and nearest_right_entity == '':
        different_right = ner[1][i]['span'][0] - pat_index
        if different_right > 0:
          nearest_right_entity = ''.join(ner[1][i]['text'])
    else:
     if re.search(n2,ner[1][i]['entity_type']) and nearest_right_entity == '':
      different_right = ner[1][i]['span'][0] - pat_index
      if different_right > 0:
        nearest_right_entity = ''.join(ner[1][i]['text'])
  if nearest_left_entity != '' and nearest_right_entity != '':
    return (nearest_left_entity,nearest_right_entity)
  else:
    return None

function for extract **pattern** from **entities (seed tuples)**

In [ ]:
def find_pattern(ner, tuples, n1, n2):
  start = 0
  stop = 0
  for i in range(len(ner[1])):
    if type(n1) == list:
     if re.search(tuples[0].replace(' ',''),''.join(ner[1][i]['text'])) and start == 0 and ner[1][i]['entity_type'] in n1:
      start = ner[1][i]['span'][1]
    else:
     if re.search(tuples[0].replace(' ',''),''.join(ner[1][i]['text'])) and start == 0 and ner[1][i]['entity_type'] == n1:
      start = ner[1][i]['span'][1]

    if type(n2) == list:
     if re.search(tuples[1].replace(' ',''),''.join(ner[1][i]['text'])) and stop == 0 and start != 0 and ner[1][i]['entity_type'] in n2:
      stop = ner[1][i]['span'][0]
    else:
     if re.search(tuples[1].replace(' ',''),''.join(ner[1][i]['text'])) and stop == 0 and start != 0 and ner[1][i]['entity_type'] == n2:
      stop = ner[1][i]['span'][0]
      
  if stop - start > 0 and stop != 0 and start != 0:
    return ''.join(ner[0][start:stop])
  else:
    return None

ex

In [ ]:
#EXPERIMENT tag NER
ex ="เนวิน ชิดชอบ  () ปัจจุบันเป็นประธานบริหาร สโมสรฟุตบอลบุรีรัมย์ ยูไนเต็ด  อดีตเคยเป็นสมาชิกสภาผู้แทนราษฎรและรัฐมนตรีหลายสมัย และเคยเป็นหนึ่งในบุคคลใกล้ชิดคนสำคัญของอดีตนายกรัฐมนตรี  ทักษิณ ชินวัตร  และเป็นบุคคลสำคัญที่ทำให้นาย อภิสิทธิ์ เวชชาชีวะ  ได้ดำรงตำแหน่งนายกรัฐมนตรี ในปี พ.ศ. 2551"
try:
  ner = nner.get_tag(ex)
except:
  try:
    ner= nner.get_tag(ex[:1250],show_result = False)
  except:
      try:
        ner= nner.get_tag(ex[:1200])
      except:
        try:
          ner= nner.get_tag(ex[:1150])
        except:
          try:
            ner= nner.get_tag(ex[:1100])
          except:
            ner= nner.get_tag(ex[:1050])
ner

(['<s>',
  '',
  'เนวิน',
  '',
  '',
  'ชิด',
  'ชอบ',
  '',
  '(',
  ')',
  '',
  'ปัจจุบัน',
  '',
  'เป็น',
  'ประธาน',
  '',
  'บริหาร',
  '',
  '',
  'สโมสร',
  '',
  'ฟุตบอล',
  '',
  'บุรีรัมย์',
  '',
  '',
  'ยูไนเต็ด',
  '',
  'อดีต',
  'เคย',
  '',
  'เป็น',
  '',
  'สมาชิกสภาผ',
  'ู้แทนราษฎร',
  'และ',
  '',
  'รัฐมนตรี',
  '',
  'หลาย',
  '',
  'สมัย',
  '',
  'และ',
  'เคย',
  '',
  'เป็นหนึ่ง',
  '',
  'ใน',
  '',
  'บุคคล',
  '',
  'ใกล้ชิด',
  '',
  'คนสําคัญ',
  '',
  'ของ',
  'อดีต',
  'นาย',
  'กรัฐมนตรี',
  '',
  '',
  'ทักษิณ',
  '',
  '',
  'ชินวัตร',
  '',
  'และ',
  '',
  'เป็น',
  '',
  'บุคคล',
  '',
  'สําคัญ',
  '',
  'ที่',
  'ทําให้',
  'นาย',
  '',
  '',
  'อภิสิทธิ์',
  '',
  '',
  'เวช',
  '',
  'ช',
  'า',
  '',
  'ชีวะ',
  '',
  '',
  'ได้',
  '',
  'ดํารง',
  'ตําแหน่ง',
  'นาย',
  'กรัฐมนตรี',
  '',
  '',
  'ใน',
  '',
  'ปี',
  '',
  '',
  'พ',
  '.',
  'ศ',
  '.',
  '',
  '',
  '2551',
  '</s>'],
 [{'text': ['', 'เนวิน'], 'span': [1, 3], 'entit

In [ ]:
#ทดลอง
find_tuples(ner,'ประธาน',['person','orgcorp'],['org_political','orgcorp'])

In [ ]:
#ทดลอง
find_pattern(ner,('เจฟฟรีย์','วอลต์ดิสนีย์'),['person','orgcorp'],['org_political','orgcorp'])

# **Rule-based Bootstrap**


In [ ]:
!gdown 1aHF_Q-l6wxaDZwzbUkdDXSapMiPRbkaM
!unzip th_wiki.zip
!rm th_wiki.zip

Downloading...
From: https://drive.google.com/uc?id=1aHF_Q-l6wxaDZwzbUkdDXSapMiPRbkaM
To: /content/th_wiki.zip
100% 138M/138M [00:00<00:00, 234MB/s]
Archive:  th_wiki.zip
  inflating: th_wiki.txt             
  inflating: __MACOSX/._th_wiki.txt  


pat_role1 = '[person].*ทำงานเป็น.*[role]'

pat_role2 = '[person].*ดำรงตำแหน่งเป็น.*[role]'

pat_role3 = '[person].*มีอาชีพเป็น.*[role]'

pat_city_of_birth1 = '[person].*(เกิด|กำเนิด|ประสูติ)(ใน|ที่).*[city]'

pat_city_of_birth2 = '[person].*เป็นชาวเมือง.*[city]'

pat_city_of_birth3 = '[person].*บ้านเกิดอยู่ที่เมือง.*[city]'

pat_child_of1 = '[person].*มีบุตรธิดา \d คน ดังนี้ .*[person]'

pat_child_of2 = '[person].*เป็น(บุตร|ลูก)(ชาย|สาว)?ของ.*[person]'

pat_child_of3 = '[person].*เป็นทายาทของ.*[person]'

In [ ]:
pat_role1_set = set()
pat_role2_set = set()
pat_role3_set = set()
pat_work_at1_set = set()
pat_work_at2_set = set()
pat_work_at3_set = set()
pat_child_of1_set = set()
pat_child_of2_set = set()
pat_child_of3_set = set()

In [ ]:
#role
# pat_role1 = 'ทำงานเป็น'
# pat_role2 = 'ดำรงตำแหน่งเป็น'
# pat_role3 = 'มีอาชีพเป็น'
pat_role1 = 'ปัจจุบันเป็น'
pat_role2 = 'เป็น'
pat_role3 = 'จึงกลายเป็น'
#management
pat_work_at1 = 'เป็นเจ้าของ'
pat_work_at2 = 'พัฒนาโดย'
pat_work_at3 = 'เป็นผู้ร่วมก่อตั้ง'
#child of
pat_child_of1 = 'เจ้าของ'
pat_child_of2 = 'เป็นบุตรชายของ'
pat_child_of3 = 'เป็นทายาทของ'

# Role

In [ ]:
with open('th_wiki.txt') as text:
 for sent in text:
  #  if re.search(pat_role1,sent) and len(pat_role1_set) < 15:
  #   ner_list = get_ner(sent)
  #   if ner_list is not None:
  #    tuples = find_tuples(ner_list,'ปัจจุบัน','person','role')
  #    if tuples is not None and tuples not in pat_role1_set and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
  #      pat_role1_set.add(tuples)
  #      print('{relation}:{tuples}'.format(relation=pat_role1, tuples=tuples))
  #      print(sent)

   if re.search(pat_role2,sent) and len(pat_role2_set) < 15:
    ner_list = get_ner(sent)
    if ner_list is not None:
     tuples = find_tuples(ner_list,'เป็น','person','role')
     if tuples is not None and tuples not in pat_role2_set and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
       pat_role2_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_role2, tuples=tuples))
       print(sent)

  #  if re.search(pat_role3,sent) and len(pat_role3_set) < 15:
  #   ner_list = get_ner(sent)   
  #   if ner_list is not None:
  #    tuples = find_tuples(ner_list,'กลายเป็น','person','role')
  #    if tuples is not None and tuples not in pat_role3_set and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
  #      pat_role3_set.add(tuples)
  #      print('{relation}:{tuples}'.format(relation=pat_role3, tuples=tuples))
  #      print(sent)

In [ ]:
with open('th_wiki.txt') as text:
 for sent in text:
   if re.search(pat_role1,sent) and len(pat_role1_set) < 20:
    ner_list = get_ner(sent)
    if ner_list is not None:
     tuples = find_tuples(ner_list,'ทํางาน','person','role')
     if tuples is not None and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
       pat_role1_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_role1, tuples=tuples))
       print(sent)

   if re.search(pat_role2,sent) and len(pat_role2_set) < 20:
    ner_list = get_ner(sent)
    if ner_list is not None:
     tuples = find_tuples(ner_list,'ตําแหน่ง','person','role')
     if tuples is not None and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
       pat_role2_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_role2, tuples=tuples))
       print(sent)

   if re.search(pat_role3,sent) and len(pat_role3_set) < 20:
    ner_list = get_ner(sent)   
    if ner_list is not None:
     tuples = find_tuples(ner_list,'อาชีพ','person','role')
     if tuples is not None and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
       pat_role3_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_role3, tuples=tuples))
       print(sent)

ดำรงตำแหน่งเป็น:('พระเจ้าวิลเฮล์ม', 'จักรพรรดิเยอรมัน')
ในปี 1864 ความขัดแย้งระหว่างออสเตรียกับปรัสเซียขึ้นอีกครั้ง และบานปลายเป็นสงครามออสเตรีย-ปรัสเซีย หรือที่เรียกว่า "สงครามพี่น้อง" สงครามครั้งนี้จบลงด้วยชัยชนะอย่างงดงามของปรัสเซีย ออสเตรียสูญเสียอิทธิพลเหนือรัฐเยอรมันตอนใต้ทั้งหมดและจำยอมยุบสมาพันธรัฐเยอรมันในวันที่ 23 สิงหาคม 1866 และนำไปสู่การสถาปนา "สมาพันธรัฐเยอรมันเหนือ" ที่มีกษัตริย์แห่งปรัสเซียเป็นองค์ประธาน และภายหลังปรัสเซียมีชัยในสงครามฝรั่งเศส-ปรัสเซียในปี 1871 รัฐสภาแห่งสมาพันธรัฐเยอรมันเหนือได้มีมติให้เปลี่ยนชื่อสมาพันธรัฐเป็นจักรวรรดิ และมีมติให้พระเจ้าวิลเฮล์มที่ 1 แห่งปรัสเซีย ดำรงตำแหน่งเป็นจักรพรรดิเยอรมัน ถือเป็นการสถาปนาจักรวรรดิเยอรมันอย่างเป็นทางการ 

ดำรงตำแหน่งเป็น:('โจไบเดิน', 'ประธานาธิบดีสหรัฐ')
โจ ไบเดินจากพรรคเดโมแครต ผู้ชนะการเลือกตั้งประธานาธิบดีปี 2020 ปัจจุบันดำรงตำแหน่งเป็นประธานาธิบดีสหรัฐคนที่ 46 ผู้นำปัจจุบันในวุฒิสภา ได้แก่ รองประธานาธิบดีกมลา แฮร์ริสจากพรรคเดโมแครต ประธานชั่วคราวแพทริก ลีฮี (Patrick Leahy) จากพรรคเดโมแครต หัวหน้าฝ่ายข้างมาก ช

# Manage

In [ ]:
with open('th_wiki.txt') as text: 
 for sent in text:
   ner_list = []
   if re.search(pat_work_at1,sent) and len(pat_work_at1_set) < 40:
    ner_list = get_ner(sent)
    if ner_list is not None:
     tuples = find_tuples(ner_list,'เจ้าของ',['person','orgcorp'],['org_political','orgcorp'])
     if tuples is not None and tuples not in pat_work_at1_set and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
       pat_work_at1_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_work_at1, tuples=tuples))
       print(sent)

   if re.search(pat_work_at2,sent) and len(pat_work_at2_set) < 30:
    ner_list = get_ner(sent)
    if ner_list is not None:
     tuples = find_tuples(ner_list,'พัฒนา',['org_political','orgcorp'],['person','orgcorp'])
     if tuples is not None and tuples not in pat_work_at2_set and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
       pat_work_at2_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_work_at2, tuples=tuples))
       print(sent)

  #  if re.search(pat_work_at3,sent) and len(pat_work_at3_set) < 15:
  #   ner_list = get_ner(sent)
  #   if ner_list is not None:
  #    tuples = find_tuples(ner_list,'ก่อตั้ง',['person','orgcorp'],['org_political','orgcorp'])
  #    if tuples is not None and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
  #      pat_work_at3_set.add(tuples)
  #      print('{relation}:{tuples}'.format(relation=pat_work_at3, tuples=tuples))
  #      print(sent)

เป็นเจ้าของ:('ซิลวีโอแบร์ลุสโกนี', 'มีเดียเซต')
ซิลวีโอ แบร์ลุสโกนี (, เกิด 29 กันยายน ค.ศ. 1936 ที่นครมิลาน) เป็นนักการเมืองและนักธุรกิจชาวอิตาลี ซึ่งดำรงตำแหน่งนายกรัฐมนตรีอิตาลีสามสมัย ตั้งแต่ ค.ศ. 1994-1995, 2001-2006 และ 2008-2011 เขายังเป็นเจ้าของมีเดียเซตและประธานสโมสรฟุตบอลเอซี มิลาน 

เป็นเจ้าของ:('ฟอร์ซาอิตาเลีย', 'บริษัทฟินอินเวสต์จักรวรรดิสื่อสารมวลชน')
ซิลวีโอ แบร์ลุสโกนีเป็นหัวหน้าพรรคการเมืองชื่อว่า ฟอร์ซา อิตาเลีย (ที่เขาได้จัดตั้งขึ้นเป็นการพิเศษ เพื่อเป็นการปูทางเข้าสู่ถนนทางการเมืองของตนเอง ที่เขาอ้างว่าเป็นการกระทำ"โดยชอบธรรม") และยังเป็นเจ้าของบริษัทฟินอินเวสต์ จักรวรรดิสื่อสารมวลชนในประเทศอิตาลี (ที่ควบคุมกิจการของมีเดียเซ็ตอีกต่อหนึ่ง) ในฐานะผู้ประกอบการ เขามีความสนใจในธุรกิจหลากหลายประเภท และสามารถควบคุมสื่อส่วนใหญ่ในประเทศอิตาลีได้ โดยไม่ทางตรงก็ทางอ้อม (โดยเฉพาะอย่างยิ่งสถานีโทรทัศน์หกช่องในประเทศ ที่ สามช่องเป็นของเอกชน และสามช่องเป็นของรัฐ โดยที่เขาสามารถควบคุมสามช่องหลังได้ในฐานะหัวหน้ารัฐบาล) ในการจัดอันดับมหาเศรษฐีโลกของนิตยสารฟอร์บ ได้ระบุว่าเขาเป็นบุคคล

# Child of

In [ ]:
with open('th_wiki.txt') as text:
 for sent in text:
   if re.search(pat_child_of1,sent) and len(pat_child_of1_set) < 10:
    ner_list = get_ner(sent)
    if ner_list is not None:
     tuples = find_tuples(ner_list,'บุตร','person','person')
     if tuples is not None and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
      #  tuples = (tuples[1],tuples[0])      
       pat_child_of1_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_child_of1, tuples=tuples))
       print(sent)

   if re.search(pat_child_of2,sent) and len(pat_child_of2_set) < 10:
    ner_list = get_ner(sent)
    if ner_list is not None:
     tuples = find_tuples(ner_list,'บุตร','person','person')
     if tuples is not None and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
       pat_child_of2_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_child_of2, tuples=tuples))
       print(sent)

   if re.search(pat_child_of3,sent) and len(pat_child_of3_set) < 10:
    ner_list = get_ner(sent)
    if ner_list is not None:
     tuples = find_tuples(ner_list,'ทายาท','person','person')
     if tuples is not None and tuples[0] != tuples[1] and tuples[0] != '<unk>' and tuples[1] != '<unk>':
       pat_child_of3_set.add(tuples)
       print('{relation}:{tuples}'.format(relation=pat_child_of3, tuples=tuples))
       print(sent)

มีบุตรธิดา.*คือ :('สกุลเดิมศกุนตาภัย', 'ปรางเวชชาชีวะ')
อภิสิทธิ์ เวชชาชีวะ สมรสกับรองศาสตราจารย์ พิมพ์เพ็ญ เวชชาชีวะ (สกุลเดิม ศกุนตาภัย) อดีตทันตแพทย์ ผู้ช่วยศาสตราจารย์ประจำภาควิชาคณิตศาสตร์ คณะวิทยาศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย มีบุตรธิดาด้วยกัน 2 คนคือ ปราง เวชชาชีวะ และ ปัณณสิทธิ์ เวชชาชีวะ ปัณณสิทธิ์นั้นเป็นโรคออทิซึมมาแต่กำเนิด ศาลเยาวชนและครอบครัวกลางจึงสั่งให้เป็นคนเสมือนไร้ความสามารถ และให้อยู่ในความพิทักษ์ของอภิสิทธิ์ผู้เป็นบิดาตั้งแต่วันที่ 3 กันยายน พ.ศ. 2555โดยบุตรชายของ อภิสิทธิ์ เวชชาชีวะ เรียนที่ โรงเรียนสาธิตแห่งมหาวิทยาลัยเกษตรศาสตร์ นาย อภิสิทธิ์ เวชชาชีวะ จึงได้เข้าร่วม เป็น สมาชิก ชมรมผู้ปกครองเพื่อเด็กพิเศษ โรงเรียนสาธิตแห่งมหาวิทยาลัยเกษตรศาสตร์ เช่นเดียวกับ ทนง พิทยะ

เป็นบุตรชายของ:('โยฮันชเตราส์', 'โยฮันชเตราส์')
โยฮัน ชเตราส์ ผู้บุตร เป็นบุตรชายของโยฮัน ชเตราส์ ผู้พ่อ ซึ่งเป็นคีตกวีเช่นกัน น้องชายของเขา 2 คน คือ โยเซฟ ชเตราส์ กับ เอด๊วด ชเตราส์ ก็เป็นนักประพันธ์เพลง แต่โยฮันที่สองเป็นคนที่โด่งดังที่สุดในตระกูล เขาเป็นที่รู้จักในนามของ "ราชาเพลงวอลซ์" ในช่วงท

In [ ]:
with open('th_wiki.txt') as text:
 for sent in text:
   if re.match(pat_role1,sent) or re.match(pat_role2,sent) or re.match(pat_role3,sent) or re.match(pat_city_of_birth1,sent) or re.match(pat_city_of_birth2,sent) or re.match(pat_city_of_birth3,sent) or re.match(pat_child_of1,sent) or re.match(pat_child_of2,sent) or re.match(pat_child_of3,sent):
    try:
      ner_list = nner.get_tag(sent, show_result = True)
    except:
      continue

<s>||ดํารง|ตําแหน่ง||เป็น||ศ|าสตราจารย์||พิเศษ|||ภาควิชา||ภาษา||ตะวันออก|||จุฬาลงกรณ์||มหาวิทยาลั|ย||และ||ดํารง|ตําแหน่ง||เป็น||ภาคี||สมาชิก|||สาขา||ศาสน|ศาสตร์|||สํานัก||ธรรมศาสตร์|และ||การเมือง|||ราช|บัณฑิต|ย|สถาน||</s> 

[12, 18]       org_edu        ภาควิชาภาษาตะวันออก
[19, 24]       org_edu        จุฬาลงกรณ์มหาวิทยาลัย
[19, 24]       org_edu        จุฬาลงกรณ์มหาวิทยาลัย
[36, 55]       org_edu        สาขาศาสนศาสตร์สํานักธรรมศาสตร์และการเมืองราชบัณฑิตยสถาน
[42, 46]       org_edu        สํานักธรรมศาสตร์
[44, 46]       org_edu        ธรรมศาสตร์
<s>||ทํางาน||เป็น|อาจารย์||ประจํา||ภาควิชา||ฟิสิกส์|||คณะวิทยา|ศาสตร์|||จุฬาลงกรณ์||มหาวิทยาลั|ย|||ปี|||พ|.|ศ|.|||25|13||หัวหน้า||ภาควิชา||ค|อมพิวเตอร์||ธุรกิจ|||วิทยาลัย||อัสสัมชัญ||บริหาร|ธุรกิจ|||ใน||ปี|||พ|.|ศ|.|||2521|||เป็น||กรรมการ||วิชาการ|||จุฬาลงกรณ์||มหาวิทยาลั|ย|||ใน||ปี|||พ|.|ศ|.|||2526|||ทํางาน||เป็น||กรรมการ|และ||เป็นหนึ่ง||ใน||ผู้ก่อตั้ง|ศูนย์||เทคโนโลยี||อิเล็กทรอน|ิกส์|และ||ค|อมพิวเตอร์||แห่งชาติ|||หรือ|||เน|ค|เทค|||ขึ้น||ใน||

# **Find more pattern #1**

In [ ]:
print('{a}: {b}'.format(a = pat_role1, b = pat_role1_set))
print('{a}: {b}'.format(a = pat_role2, b = pat_role2_set))
print('{a}: {b}'.format(a = pat_role3, b = pat_role3_set))

ทำงานเป็น: {('สนธิ', 'บรรณาธิการบริหารหนังสือพิมพ์"ประชาธิปไตย'), ('ยูลียา', 'ประธานบริษัทกิจการพลังงาน'), ('แทตเชอร์', 'สมาชิกสภาผู้แทนราษฎร'), ('บ็อลท์ซาโน', 'หัวหน้าภาควิชาปรัชญา'), ('โจนินที', 'หัวหน้าหน่วยลับ(อันบุ'), ('ชิซุรุ', 'รูมเมท'), ('จุงโกะอิวาโอะ', 'อดีตสมาชิกวง"<unk>'), ('สุริยะใส', 'นักวิชาการวุฒิสภา'), ('ไอลีน', 'ผู้ช่วยศาสตราจารย์สาขาคณิตศาสตร์'), ('มาริก้า', 'บาทหลวง')}
ดำรงตำแหน่งเป็น: {('ประภัสร์', 'ผู้ว่าราชการกรุงเทพมหานคร'), ('พระเจ้าวิลเฮล์ม', 'จักรพรรดิเยอรมัน'), ('อภิสิทธิ์', 'รัฐมนตรีประจําสํานักนายกรัฐมนตรี'), ('<unk>arrister<unk>uhammad<unk>amiruddin<unk>ircar', 'รักษาการประธานาธิบดี'), ('เจ้าธรรมเทโว', 'ตากะจะเจ้าเมืองขุขันธ์'), ('บรรหาร', 'นายกรัฐมนตรี'), ('ร.ต.ท.พวงศรีบุญลือ', 'นายอําเภอยะโสธร'), ('โจไบเดิน', 'ประธานาธิบดีสหรัฐ'), ('พลเอกสุรยุทธ์จุลานนท์', 'นายกรัฐมนตรี'), ('ฯพณฯพลเอกเปรมติณสูลานนท์', 'นายกรัฐมนตรี')}
มีอาชีพเป็น: {('คลินตัน', 'ประธานาธิบดี'), ('อัลเฟรดเลนนอน', 'นายกรัฐมนตรีสหราชอาณาจักร'), ('<unk>rancesco<unk>iamberti', 'สถาปนิก'), ('ภ

In [ ]:
print('{a}: {b}'.format(a = pat_work_at1, b = pat_work_at1_set))
print('{a}: {b}'.format(a = pat_work_at2, b = pat_work_at2_set))
print('{a}: {b}'.format(a = pat_work_at3, b = pat_work_at3_set))

บริหารโดย: {('แอลวีเอ็มเอช', 'อาแล็กซ็องดร์อาร์โน'), ('จิฟฟี่', 'บริษัทปตท.บริหารธุรกิจค้าปลีกจํากัด'), ('ไอบีซีแคมโบเดีย', 'พรรคประชาชนกัมพูชา'), ('เซ็นทรัลแฟมิลี่มาร์ท', 'เซ็นทรัลเรสเทอรองค์กรุ๊ป'), ('ไฟร์ฟอกซ์', 'บริษัทมอซิลลา'), ('วัน-ทู-โกแอร์ไลน์', 'โอเรียนท์ไทยแอร์ไลน์'), ('กัดดาฟี', "<unk>eneral<unk>eople's<unk>ongress"), ('แมริออทอินเตอร์เนชั่นแนล', 'แมริออท'), ('เบลล์เฮลิคอปเตอส์', 'บริษัทเคเอชไอเมเนจเมนต์จํากัด'), ('อีจีวี', 'ซีนีเพล็กซ์')}
เป็นประธานของ: {('คาร์ลเทโอดอร์ฟอนดัลแบร์ก', 'คณะพระมหากษัตริย์'), ('สแควร์เอนิกซ์', 'บริษัทสแควร์'), ('เจฟฟรีย์แคตเซนเบิร์ก', 'บริษัทวอลต์ดิสนีย์'), ('นักธุรกิจฌอนพาร์กเกอร์', 'เฟซบุ๊ก'), ('สมเด็จกรมพระนโรดมรณฤทธิ์', 'พรรคฟุนซินเปก'), ('บิลเกตส์', 'พอร์ตแลนด์เทรลเบลเซอร์ส'), ('เดวิดแบร์รี่ชาร์ลส์ดีน', 'บริษัทธุรกิจเรดแอนด์ไวท์'), ('ซู', 'พรรคสมัชชาแห่งชาติแอฟริกา'), ('อัลเบิร์ตอาร์โนลด์"อัล"กอร์จูเนียร์', 'บริษัทเจเนอเรชั่นอินเวสท์เมนท์แมเนจเมนท์'), ('โอกูโบะโทชิมิจิ', '<unk>ssemblyof<unk>refectural<unk>overnors')}
เป็นผู้บริหารบริษัท: {

In [ ]:
print('{a}: {b}'.format(a = pat_child_of1, b = pat_child_of1_set))
print('{a}: {b}'.format(a = pat_child_of2, b = pat_child_of2_set))
print('{a}: {b}'.format(a = pat_child_of3, b = pat_child_of3_set))

ทำงานเป็น: {('ชิซุรุ', 'รูมเมท'), ('สนธิ', 'บรรณาธิการบริหารหนังสือพิมพ์"ประชาธิปไตย'), ('ไอลีน', 'ผู้ช่วยศาสตราจารย์สาขาคณิตศาสตร์'), ('ยูลียา', 'ประธานบริษัทกิจการพลังงาน'), ('โจนินที', 'หัวหน้าหน่วยลับ(อันบุ')}
ดำรงตำแหน่งเป็น: {('พลเอกสุรยุทธ์จุลานนท์', 'นายกรัฐมนตรี'), ('พระเจ้าวิลเฮล์ม', 'จักรพรรดิเยอรมัน'), ('<unk>arrister<unk>uhammad<unk>amiruddin<unk>ircar', 'รักษาการประธานาธิบดี'), ('อภิสิทธิ์', 'รัฐมนตรีประจําสํานักนายกรัฐมนตรี'), ('โจไบเดิน', 'ประธานาธิบดีสหรัฐ')}
มีอาชีพเป็น: {('ตาทอง', 'ครูนายโรงละครหลวง'), ('โมเชดายัน', 'ผู้สื่อข่าวสงคราม'), ('<unk>rancesco<unk>iamberti', 'สถาปนิก'), ('อัลเฟรดเลนนอน', 'นายกรัฐมนตรีสหราชอาณาจักร'), ('คลินตัน', 'ประธานาธิบดี')}
เกิดในเมือง: {('หลินซินหรู', 'เมืองไทเป'), ('บิลคลินตัน', 'โฮป'), ('เลนิน', 'เมืองซิมบิร์สค์'), ('<unk>eorge<unk>astman', 'เมืองวอเทอร์วิลล์'), ('โอมาร์คัยยาม', 'เมืองเนชาปูร')}
เป็นชาวเมือง: {('โจวเอินไหล', 'มณฑลเจ้อเจียง'), ('จ้งหนี', 'หลู่'), ('สตีเวนแพดด็อก', 'เมสกีต'), ('สมเด็จกรมพระยาดํารงราชานุภาพ', 'เมืองลับ

In [ ]:
new_pat_role = {'ทำงานเป็น','ดำรงตำแหน่งเป็น','มีอาชีพเป็น'}
new_pat_work_at = {'บริหารโดย','เป็นประธานของ','เป็นผู้บริหารบริษัท'}
new_pat_child_of = {'เป็นบุตรชายของ','เป็นทายาทของ'}

In [ ]:
seed_tuples_role1 = ('โจไบเดิน','ประธานาธิบดี')
seed_tuples_role2 = ('อภิสิทธิ์','นายก')
seed_tuples_role3 = ('อิเอยาซุ','โชกุน')
seed_tuples_work_at1 = ('บิลเกตส์','ไมโครซอฟท์')
seed_tuples_work_at2 = ('คุณหญิงสุดารัตน์','พรรคเพื่อไทย')
seed_tuples_work_at3 = ('ธรัชศุภโชคไพศาล','ไอยรา')
seed_tuples_child_of1 = ('ณรงค์กิตติขจร','ถนอมกิตติขจร')
seed_tuples_child_of2 = ('อภิสิทธิ์','ปัณณสิทธิ์')
seed_tuples_child_of3 = ('','')

# new pattern of ROLE relation

In [ ]:
with open('th_wiki.txt') as text:
 limit = 0
 for sent in text:
   if re.search(seed_tuples_role1[0],sent.replace(' ','')) and re.search(seed_tuples_role1[1],sent.replace(' ','')) and limit <= 50:
    ner_list = get_ner(sent)
    new_pattern = find_pattern(ner_list,seed_tuples_role1,'person','role')
    if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_role and len(new_pattern) <= 100:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_role.add(new_pattern)

new pattern: จากพรรคเดโมแครตผู้ชนะการเลือกตั้ง
context: โจ ไบเดินจากพรรคเดโมแครต ผู้ชนะการเลือกตั้งประธานาธิบดีปี 2020 ปัจจุบันดำรงตำแหน่งเป็นประธานาธิบดีสหรัฐคนที่ 46 ผู้นำปัจจุบันในวุฒิสภา ได้แก่ รองประธานาธิบดีกมลา แฮร์ริสจากพรรคเดโมแครต ประธานชั่วคราวแพทริก ลีฮี (Patrick Leahy) จากพรรคเดโมแครต หัวหน้าฝ่ายข้างมาก ชัก ชูเมอร์ (Chuck Schumer) และหัวหน้าฝ่ายข้างน้อย มิตช์ แม็กคอนเนล (Mitch McConnell) ผู้นำในสภาผู้แทนราษฎร ได้แก่ ประธานสภาผู้แทนราษฎรแนนซี เพโลซี หัวหน้าฝ่ายข้างมาก สเตนี ฮอยเยอร์ (Steny Hoyer) และหัวหน้าฝ่ายข้างน้อย เควิน แม็กคาธี (Kevin McCarthy)

new pattern: เป็นนักการเมืองชาวอเมริกันและเป็น
context: โจเซฟ โรบิเนตต์ ไบเดิน จูเนียร์ (; เกิด 20 พฤศจิกายน พ.ศ. 2485) หรือ โจ ไบเดิน เป็นนักการเมืองชาวอเมริกัน และเป็นประธานาธิบดีสหรัฐคนปัจจุบัน หลังการชนะการเลือกตั้งประธานาธิบดี พ.ศ. 2563 ในอดีตไบเดินเคยดำรงตำแหน่งเป็นรองประธานาธิบดีสหรัฐคนที่ 47 ระหว่าง พ.ศ. 2552 ถึง 2560 เป็นสมาชิกวุฒิสภาจากรัฐเดลาแวร์เจ็ดสมัยติดต่อกัน ระหว่าง พ.ศ. 2515 ถึง 2552 สังกัดพรรคเดโมแครต

new pa

In [ ]:
with open('th_wiki.txt') as text:
 for sent in text:
   limit = 0
   if re.search(seed_tuples_role2[0],sent.replace(' ','')) and re.search(seed_tuples_role2[1],sent.replace(' ','')) and limit <= 50:
    ner_list = get_ner(sent)
    new_pattern = find_pattern(ner_list,seed_tuples_role2,'person','role')
    if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_role and len(new_pattern) <= 100:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_role.add(new_pattern)
      limit += 1

new pattern: เป็น
context: กลางปี 2540 เกิด "วิกฤตต้มยำกุ้ง" ซึ่งโค่นรัฐบาลบรรหารและชวลิต ในปีเดียวกันยังมีการประกาศใช้รัฐธรรมนูญปี 2540 ซึ่งมีสมาชิกสภาร่างรัฐธรรมนูญมาจากการเลือกตั้ง หลังจากนั้นปี 2544 ทักษิณ ชินวัตรนำพรรคไทยรักไทยชนะการเลือกตั้งถล่มทลายในปี 2544 และปี 2548 แนวนโยบายประชานิยมของเขาทำให้ได้รับเสียงสนับสนุนจำนวนมากในชนบท แต่อภิชน ข้าราชการและชนชั้นกลางในเมืองคัดค้านเขา การประท้วงนำโดยกลุ่มพันธมิตรประชาชนเพื่อประชาธิปไตยในปี 2549 ตามมาด้วยรัฐประหารในปีเดียวกัน นับแต่นั้นการเมืองไทยอยู่ในวิกฤตการเมืองระหว่างผู้สนับสนุนและผู้คัดค้านทักษิณ ชินวัตร หลังมีการประกาศใช้รัฐธรรมนูญปี 2550 ก็มีการจัดการเลือกตั้งทั่วไป ผลปรากฏว่า สมัคร สุนทรเวชนำพรรคพลังประชาชนซึ่งเป็นพันธมิตรของทักษิณชนะการเลือกตั้งอีก ในปี 2551 พันธมิตรประชาชนเพื่อประชาธิปไตยชุมนุมตั้งแต่เดือนพฤษภาคมถึงธันวาคม จนเมื่อศาลรัฐธรรมนูญวินิจฉัยยุบพรรคพลังประชาชน ทำให้มีการรวบรวมเสียงในสภาเพื่อเลือกอภิสิทธิ์ เวชชาชีวะเป็นนายกรัฐมนตรี ในครั้งนั้นพลเอกเปรมและพลเอก อนุพงษ์ เผ่าจินดา ผู้บัญชาการทหารบก อำนวยความสะดวกหรือสั่ง

In [ ]:
with open('th_wiki.txt') as text:
 limit = 0
 for sent in text:
   if re.search(seed_tuples_role3[0],sent.replace(' ','')) and re.search(seed_tuples_role3[1],sent.replace(' ','')) and limit <= 50:
    ner_list = get_ner(sent)
    new_pattern = find_pattern(ner_list,seed_tuples_role3,'person','role')
    if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_role and len(new_pattern) <= 100:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_role.add(new_pattern)
      limit += 1

new pattern: ประกาศสละตําแหน่ง
context: ในปีค.ศ. 1867 โชกุนโทกูงาวะ โยชิโนบุ ประกาศสละตำแหน่งโชกุนและถวายอำนาจคืนแด่พระจักรพรรดิเมจิ เป็นการสิ้นสุดการปกครองของรัฐบาลโชกุนโทกูงาวะที่มีมาเป็นเวลาสองร้อยห้าสิบกว่าปี แต่ความขัดแย้งทางการเมืองและการทหาร ระหว่างฝ่ายตระกูลโทกูงาวะและฝ่ายซนโนโจอิยังคงดำเนินต่อไป จนนำไปสู่สงครามโบชิง (Boshin War)

new pattern: ()เป็น
context: โทกูงาวะ อิเอมิตสึ () เป็น โชกุน ลำดับที่ 3 ของ ตระกูลโทกูงาวะ เขาเป็นบุตรชายคนโตของ โทกูงาวะ ฮิเดตาดะ กับ โอเอโยะ และเป็นหลานปู่ของ โทกูงาวะ อิเอยาซุ ท่านหญิงคะซุงะ เป็นแม่นม ทำหน้าที่เป็นที่ปรึกษาทางการเมืองของเขาและอยู่ในระดับแนวหน้าของการเจรจาระหว่างรัฐบาลโชกุนกับ ราชสำนักเกียวโต อิเอมิตสึปกครองตั้งแต่ ปี ค.ศ. 1623 ถึง ปี ค.ศ. 1651 ในช่วงเวลานี้เขาตรึงกางเขนชาวคริสเตียนขับไล่ชาวยุโรปทั้งหมดออกจากญี่ปุ่นและปิดพรมแดนของประเทศ นโยบายการเมืองต่างประเทศดำเนินมายาวนานกว่า 200 ปี นับตั้งแต่ก่อตั้งรัฐบาล

new pattern: ()
context: เท็นโชอิง () หรือที่รู้จักกันในนามว่า "อัตสึฮิเมะ" () เป็นมิไดโดโกโระหรือภรรยาเอกของของโชกุนโทกูงา

In [ ]:
new_pat_role

{'',
 '()',
 '()เป็น',
 '),อดีตรัฐมนตรีช่วยว่าการกระทรวงการคลัง(ในรัฐบาลนายสมัครสุนทรเวชและรัฐบาลนายสมชายวงศ์สวัสดิ์)และ',
 ')โดยมีพระบรมราชโองการโปรดเกล้าโปรดกระหม่อมแต่งตั้งนายสมชายเป็น',
 ')ในสมัยรัฐบาลชวน2(พ.ศ.2540-พ.ศ.2543)ต่อมาในปลายปีพ.ศ.2551พรรคประชาธิปัตย์สามารถเป็นแกนนําจัดตั้งรัฐบาลนายสาทิตย์วงศ์หนองเตยได้รับการแต่งตั้งให้ดํารงตําแหน่ง',
 ']]แถลงว่านี่เป็น"มาตรการตอบโต้ทางทูตอย่างแรก"ต่อการแต่งตั้งนี้เขายังแถลงอีกว่ากัมพูชากําลังแทรกแซงกิจการภายในและส่งผลให้มีการทบทวนความตกลงความร่วมมือสองฝ่ายอีกครั้งฝ่ายรัฐบาลกัมพูชาแถลงว่ารัฐบาลกัมพูชาปฏิเสธคําขอส่งตัวทักษิณข้ามแดนจากไทยด้วยเหตุผลว่ารัฐบาลกัมพูชามองว่าทักษิณเป็นเหยื่อของการกลั่นแกล้งทางการเมืองหลายเดือนก่อนการตัดสินใจของกัมพูชากําลังทหารทั้งสองประเทศปะทะกับเหนืออาณาเขตที่ทั้งสองฝ่ายอ้างสิทธิ์ติดกับ[[ปราสาทพระวิหาร]]จนทําให้ความสัมพันธ์ระหว่างทั้งสองเสื่อมลงวันที่5พฤศจิกายนเวลา20.30น.ตามเวลาท้องถิ่นกัมพูชากัมพูชาได้ประกาศว่าได้เรียกเอกอัครราชทูตของตนกลับจากประเทศไทยเป็นมาตรการตอบโต้ซกอานสมาชิกคณะรัฐมนตรีและ',
 'กู้ยืมเงินมา

# new pattern of WORK_AT relation

In [ ]:
with open('th_wiki.txt') as text:
 limit = 0
 for sent in text:
   if re.search(seed_tuples_work_at1[0],sent.replace(' ','')) and re.search(seed_tuples_work_at1[1],sent.replace(' ','')) and limit <= 50:
    ner_list = get_ner(sent)
    if ner_list is not None:
     new_pattern = find_pattern(ner_list,seed_tuples_work_at1,['person','orgcorp'],['org_political','orgcorp'])
     if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_work_at and len(new_pattern) <= 150:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_work_at.add(new_pattern)
      limit+=1

new pattern: ออกประกาศภายในเรื่องคลื่นแห่งระบบอินเทอร์เน็ต,
context: วันที่ 26 พฤษภาคม ค.ศ. 1995 บิล เกตส์ ออกประกาศภายในเรื่อง คลื่นแห่งระบบอินเทอร์เน็ต, ไมโครซอฟท์เริ่มต้นสายการผลิตสำหรับผลิตภัณฑ์สำหรับระบบเครือข่าย ในวันที่ 24 สิงหาคม ค.ศ. 1995 ก็ได้เปิดตัวบริการออนไลน์อย่างเอ็มเอสเอ็น ซึ่งเป็นคู่แข่งโดยตรงของเอโอแอล โดยเอ็มเอสเอ็นให้บริการเกี่ยวกับข้อมูลออนไลน์ของไมโครซอฟท์บริษัทยังคงสาขาไปตลาดใหม่ และในปี ค.ศ. 1996 เริ่มมีกิจการเคเบิลทีวีของตัวเองซึ่งใช้ชื่อว่า เอ็นบีซี โดยเป็นสถานีเคเบิลทีวีแบบ 24/7 ไมโครซอฟท์ได้เข้าสู่วงการพีดีเอ ด้วย วินโดวส์ ซีอี 1.0 ซึ่งเป็นระบบปฏิบัติการที่ออกแบบมาสำหรับเครื่องพีดีเอ ใช้หน่วยความจำและสมรรถนำต่ำ เช่น handhelds และคอมพิวเตอร์ขนาดเล็กอื่นๆ 

new pattern: เจ้าของ
context: นอกจากนี้ยังมีมหาเศรษฐี บิล เกตส์ เจ้าของบริษัทคอมพิวเตอร์ไมโครซอฟท์ (ดรอปเรียนในช่วงปริญญาตรี ขณะอายุ 19 ปี เพื่อไปสานต่อความฝันทางธุรกิจ จนกระทั่งประสบความสำเร็จ และต่อมาทางมหาวิทยาลัยฮาร์วาร์ดได้ประสาทปริญญาดุษฎีบัณฑิตกิตติมศักดิ์(วุฒิปริญญาดุษฎีบัณฑิต จำกัดสาขาหรือผู้นำสาขา

In [ ]:
with open('th_wiki.txt') as text:
 limit = 0
 for sent in text:
   if re.search(seed_tuples_work_at2[0],sent.replace(' ','')) and re.search(seed_tuples_work_at2[1],sent.replace(' ','')) and limit <= 50:
    ner_list = get_ner(sent)
    if ner_list is not None:
     new_pattern = find_pattern(ner_list,seed_tuples_work_at2,['person','orgcorp'],['org_political','orgcorp'])
     if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_work_at and len(new_pattern) <= 100:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_work_at.add(new_pattern)
      limit+=1

new pattern: (ชื่อเล่น:หน่อย;เกิด:1พฤษภาคมพ.ศ.2504)เป็นนักการเมืองสตรีชาวไทยอดีตสมาชิก
context: คุณหญิงสุดารัตน์ เกยุราพันธุ์ (ชื่อเล่น: หน่อย; เกิด: 1 พฤษภาคม พ.ศ. 2504) เป็นนักการเมืองสตรีชาวไทย อดีตสมาชิกพรรคเพื่อไทย เคยร่วมประท้วงในเหตุการณ์พฤษภา 2535 อดีตรัฐมนตรีเคยดำรงตำแหน่งที่สำคัญทั้งในรัฐบาล ทักษิณ ชินวัตร และ ชวน หลีกภัย ปัจจุบันเป็นประธานมูลนิธิไทยพึ่งไทย และประธานกรรมการผู้ก่อตั้งพรรคไทยสร้างไทย อีกทั้ง ยังสนิทสนมกับ อภิรักษ์ โกษะโยธิน นักการเมืองหลายสมัย

new pattern: แกนนํา
context: เดือนกุมภาพันธ์ พ.ศ. 2562 พลเอก อภิรัชต์ คงสมพงษ์ ผู้บัญชาการทหารบก อ้างถึงเพลงนี้เมื่อคุณหญิงสุดารัตน์ เกยุราพันธุ์ แกนนำพรรคเพื่อไทย ประกาศนโยบายการลดงบประมาณการจัดซื้ออาวุธของกองทัพ และปฏิรูปทหารเกณฑ์ให้ใช้ระบบทหารอาชีพที่มีการคัดเลือกตามความรู้ความสามารถด้วยความสมัครใจ นอกจากนี้ ยังมีกระแสข่าวว่า พลเอกอภิรัชต์สั่งการให้สถานีวิทยุกระจายเสียงเครือข่ายกองทัพบก นำเพลงนี้ รวมถึงเพลงปลุกใจอื่น ๆ อาทิ มาร์ชกองทัพบก ความฝันอันสูงสุด ไปเปิดในช่วงเวลา 07.00 น. และ 12.00 น. #หนักแผ่นดิน ขึ้นอันดับ 1

In [ ]:
with open('th_wiki.txt') as text:
 limit=0
 for sent in text:
   if re.search(seed_tuples_work_at3[0],sent.replace(' ','')) and re.search(seed_tuples_work_at3[1],sent.replace(' ','')) and limit <= 50:
    ner_list = get_ner(sent)
    if ner_list is not None:
     new_pattern = find_pattern(ner_list,seed_tuples_work_at3,['person','orgcorp'],['org_political','orgcorp'])
     if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_work_at and len(new_pattern) <= 150:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_work_at.add(new_pattern)
      limit+=1

In [ ]:
new_pat_work_at

{'(ชื่อเล่น:หน่อย;เกิด:1พฤษภาคมพ.ศ.2504)เป็นนักการเมืองสตรีชาวไทยอดีตสมาชิก',
 '(ชื่อเล่น:โจหรือโจไอยรา)เป็นผู้อํานวยการสร้างภาพยนตร์ผู้กํากับภาพยนตร์ชาวไทยและเป็นผู้บริหารบริษัท',
 ',นายชวนหลีกภัยและนายอภิสิทธิ์เวชชาชีวะชีวิตส่วนตัวสมรสกับนายจุลพันธ์อมรวิวัฒน์ส.ส.จังหวัดเชียงใหม่',
 'บริหารโดย',
 'ปัจจุบันเป็นผู้บริหาร',
 'รวมถึงเป็นผู้ร่วมก่อตั้งและเป็นประธานร่วมของมูลนิธิบิลและเมลินดาเกตส์ตลอดจนเป็นอดีตผู้จัดการหน่วยในหลายแผนกผลิตภัณฑ์ของ',
 'อดีตประธานยุทธศาสตร์',
 'อดีตประธานยุทธศาสตร์การเลือกตั้งของ',
 'ออกประกาศภายในเรื่องคลื่นแห่งระบบอินเทอร์เน็ต,',
 'เจ้าของ',
 'เปิดตัวในฐานะสมาชิกวงเกิร์ลกรุป<unk>e-<unk>ซึ่งประกอบด้วยสมาชิก8คนที่บริหารโดย',
 'เป็นประธานของ',
 'เป็นผู้บริหารบริษัท',
 'แกนนํา',
 'แต่อีกหลายเดือนให้หลังก็ย้ายกลับเข้า',
 'โดยเริ่มงานการเมืองด้วยการเป็นทีมนโยบายของ'}

# new pattern of CHILD_OF relation

In [ ]:
with open('th_wiki.txt') as text:
 limit =0 
 for sent in text:
   if re.search(seed_tuples_child_of1[0],sent.replace(' ','')) and re.search(seed_tuples_child_of1[1],sent.replace(' ','')) and limit <=50:
    ner_list = get_ner(sent)
    if ner_list is not None:
     new_pattern = find_pattern(ner_list,seed_tuples_child_of1,'person','person')
     if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_child_of and len(new_pattern) <= 150:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_child_of.add(new_pattern)
      limit+=1

new pattern: ภายหลังเหตุการณ์
context: จอมพล ถนอม กิตติขจร ( 11 สิงหาคม พ.ศ.2454 – 16 มิถุนายน พ.ศ.2547 ) เป็นนายกรัฐมนตรีไทยคนที่ 10 เคยดำรงตำแหน่งผู้บัญชาการทหารบก ผู้บัญชาการทหารสูงสุด รัฐมนตรีว่าการกระทรวงกลาโหม และรัฐมนตรีว่าการกระทรวงการต่างประเทศ ได้ชื่อว่าหนึ่งใน " สามทรราช " เป็นหัวหน้าคณะรัฐบาลในเหตุการณ์ 14 ตุลา พ.ศ.2516 ซึ่งเป็นเหตุการณ์การประท้วงของ นิสิต นักศึกษา และประชาชน โดยในเหตุการณ์ ทหารได้ใช้อาวุธสงครามเข้าปราบปรามนักศึกษาประชาชนที่ชุมนุมเรียกร้องรัฐธรรมนูญ จนทำให้มีผู้เสียชีวิตเป็นจำนวนมาก เป็นผลให้จอมพล ถนอม กิตติขจร ต้องประกาศลาออกจากตำแหน่ง และเดินทางออกจากประเทศ พร้อมกับ จอมพล ประภาส จารุเสถียร และ พ.อ.ณรงค์ กิตติขจร ภายหลังเหตุการณ์ จอมพล ถนอม กิตติขจร ก็ได้เดินทางกลับ แล้วบวชเป็นพระสามเณร เป็นชนวนไปสู่การขับไล่ ของนักศึกษาธรรมศาสตร์ จนโยงไปสู่เหตุการณ์ 6 ตุลา พ.ศ.2519 ทำให้มีผู้บาดเจ็บ เสียชีวิต และสูญหายจำนวนมาก

new pattern: (21ตุลาคมพ.ศ.2476—)เป็นอดีตนักการเมืองและเป็นบุตรชายของอดีตนายกรัฐมนตรีไทยคนที่10
context: พ.อ.ณรงค์ กิตติขจร (21 ตุลาคม พ.ศ. 2476 —)

In [ ]:
with open('th_wiki.txt') as text:
 limit =0
 for sent in text:   
   if re.search(seed_tuples_child_of2[0],sent.replace(' ','')) and re.search(seed_tuples_child_of2[1],sent.replace(' ','')) and limit<=50:
    ner_list = get_ner(sent)
    if ner_list is not None:
     new_pattern = find_pattern(ner_list,seed_tuples_child_of2,'person','person')
     if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_child_of and len(new_pattern) <= 150:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_child_of.add(new_pattern)
      limit+=1

In [ ]:
with open('th_wiki.txt') as text:
 limit=0
 for sent in text:
   if re.search(seed_tuples_child_of3[0],sent.replace(' ','')) and re.search(seed_tuples_child_of3[1],sent.replace(' ','')) and limit<=50:
    ner_list = get_ner(sent)
    if ner_list is not None:
     new_pattern = find_pattern(ner_list,seed_tuples_child_of3,'person','person')
     if new_pattern is not None and new_pattern != '' and new_pattern not in new_pat_child_of and len(new_pattern) <= 150:
      print('new pattern: ' + new_pattern)
      print('context: ' + sent)
      new_pat_child_of.add(new_pattern)
      limit+=1

In [ ]:
new_pat_child_of

{'(21ตุลาคมพ.ศ.2476—)เป็นอดีตนักการเมืองและเป็นบุตรชายของอดีตนายกรัฐมนตรีไทยคนที่10',
 ',',
 'บุตรชายของ',
 'ภายหลังเหตุการณ์',
 'เป็นทายาทของ',
 'เป็นที่คาดหมายว่าจะเป็นผู้สืบทอดอํานาจต่อจาก',
 'เป็นบุตรชายของ'}

# **ML-based Bootstrapping**

In [ ]:
def context_between(tokens):
  return {}

def between_role(tokens):
  return {}

def word_with_entity(tokens):
  return {}

In [ ]:
def featurize(tokens): 
    feature_dict = {}
    for feature_fn in [context_between,between_role,word_with_entity]: 
       feature_dict.update(feature_fn(tokens))
    return feature_dict

In [ ]:
# lr_text_classifier1.predict(test_sent_features)

In [ ]:
{'คณะกรรมการ': 1,
 'ระดับ': 1,
 'รัฐ': 1,
 'ของ': 1,
 'พรรค': 1,
 'รีพับลิกัน': 1,
 'หลาย': 1,
 'คณะ': 1,
 'มี': 1,
 'รายงาน': 1,
 'ว่า': 1,
 'พิจารณา': 1,
 'ยกเลิก': 1,
 'การเลือกตั้ง': 1,
 'ขั้นต้น': 1,
 'และ': 1,
 'การประชุม': 1,
 'ลับ': 1,
 'ปี': 1,
 ' ': 1,
 '2563': 1,
 'ส่วน': 1,
 'อีก': 1,
 'ดำเนินการ': 1,
 'ไป': 1,
 'แล้ว': 1,
 'โดย': 1,
 'อ้าง': 1,
 'ข้อเท็จจริง': 1,
 'เคย': 1,
 'เมื่อ': 1,
 'จอร์จ': 1,
 'เอช': 1,
 '.': 1,
 'ดับเบิลยู': 1,
 'บุช': 1,
 'ลงสมัคร': 1,
 'สมัย': 1,
 'ที่สอง': 1,
 'ใน': 1,
 '2535': 1,
 '2547': 1,
 'ตามลำดับ': 1,
 'เดโมแครต': 1,
 'บิล': 1,
 'คลินตัน': 1,
 'บารัค': 1,
 'โอบามา': 1,
 'กำลัง': 1,
 'รับ': 1,
 'เลือกตั้ง': 1,
 '2539': 1,
 '2555': 1,
 '\n': 1,
 'has person': False,
 'has role': False}

In [ ]:
{'วันที่': 1,
 ' ': 1,
 '10': 1,
 'มกราคม': 1,
 'พ.ศ.': 1,
 '2545': 1,
 'นิตยสาร': 1,
 '"': 1,
 'ฟา': 1,
 'ร์': 1,
 'อิสเทิร์น': 1,
 'อี': 1,
 'โค': 1,
 'โน': 1,
 'มิ': 1,
 'กรี': 1,
 'วิว': 1,
 'ลง': 1,
 'บทความ': 1,
 'ว่า': 1,
 'พระองค์': 1,
 'ทรง': 1,
 'มี': 1,
 'ความสัมพันธ์': 1,
 'แนบแน่น': 1,
 'กับ': 1,
 'พัน': 1,
 'ตำรวจ': 1,
 'โท': 1,
 'ทักษิณ': 1,
 'ชินวัตร': 1,
 'อดีต': 1,
 'นายกรัฐมนตรี': 1,
 'ไทย': 1,
 'อย่างไรก็ดี': 1,
 'ใน': 1,
 '2553': 1,
 'หลังจากที่': 1,
 'พ้น': 1,
 'จาก': 1,
 'ตำแหน่ง': 1,
 'ไป': 1,
 'โดย': 1,
 'การรัฐประหาร': 1,
 'แล้ว': 1,
 'พลเอก': 1,
 'เปรม': 1,
 'ติณ': 1,
 'สู': 1,
 'ลา': 1,
 'นนท์': 1,
 'องคมนตรี': 1,
 'ยอมรับ': 1,
 'แน่นอน': 1,
 'เจ้าฟ้า': 1,
 'ชาย': 1,
 'ยัง': 1,
 'รักษา': 1,
 'สัมพันธภาพ': 1,
 'บางอย่าง': 1,
 'นายกฯ': 1,
 'เอาไว้': 1,
 'พบ': 1,
 'เป็นช่วง': 1,
 'ๆ"': 1,
 '12': 1,
 'ธันวาคม': 1,
 'ปี': 1,
 'เดียวกัน': 1,
 'บิ': 1,
 'ฮา': 1,
 'รี': 1,
 'เคา': 1,
 'สิ': 1,
 'กัน': 1,
 '(': 1,
 'Bilahari': 1,
 'Kausikan': 1,
 ')': 1,
 'ข้าราชการ': 1,
 'กระทรวงการต่างประเทศ': 1,
 'สิงคโปร์': 1,
 'แถลง': 1,
 'ติด': 1,
 'การพนัน': 1,
 'และ': 1,
 'ได้รับ': 1,
 'การ': 1,
 'สนับสนุน': 1,
 'ทางการเงิน': 1,
 'โอกาส': 1,
 'เสริม': 1,
 'เสี่ยง': 1,
 'ทำลาย': 1,
 'ตัวเอง': 1,
 'เขา': 1,
 'อาจ': 1,
 'คิด': 1,
 'จะ': 1,
 'ปฏิบัติ': 1,
 'เหมือน': 1,
 'ดัง': 1,
 'เป็น': 1,
 'พระ': 1,
 'สหาย': 1,
 'หรือ': 1,
 'ผู้สนับสนุน': 1,
 'เพียง': 1,
 'เพราะ': 1,
 'ส่งเสริม': 1,
 'การเงิน': 1,
 'ให้': 1,
 'แก่': 1,
 'แต่': 1,
 'ไม่': 1,
 'ชอบ': 1,
 'พระทัย': 1,
 'แบบ': 1,
 'นั้น': 1,
 'สัก': 1,
 'เท่าไร': 1,
 'has person': True,
 'has role': False}

In [ ]:
{'สุทธิ': 1,
 'ชัย': 1,
 'ยัง': 1,
 'เป็น': 1,
 'ผู้ก่อตั้ง': 1,
 ' ': 1,
 'หนังสือพิมพ์': 1,
 'ธุรกิจ': 1,
 'รายวัน': 1,
 'ฉบับ': 1,
 'แรก': 1,
 'ของ': 1,
 'ประเทศ': 1,
 'ไทย': 1,
 'คือ': 1,
 '"': 1,
 'กรุงเทพ': 1,
 'เมื่อ': 1,
 'วันที่': 1,
 '6': 1,
 'ตุลาคม': 1,
 'พ.ศ.': 1,
 '2530': 1,
 'นอกจากนี้': 1,
 'ผู้ริเริ่ม': 1,
 'วิทยุ': 1,
 'เนชั่น': 1,
 'และ': 1,
 'เป็นหนึ่ง': 1,
 'ใน': 1,
 'คณะ': 1,
 'ผู้': 1,
 'ร่วม': 1,
 'ดำเนินงาน': 1,
 'สถานีโทรทัศน์': 1,
 'ไอ': 1,
 'ทีวี': 1,
 'ปี': 1,
 '2538': 1,
 ',': 1,
 'ต่อมา': 1,
 '2543': 1,
 'จึง': 1,
 'มา': 1,
 'ก่อตั้ง': 1,
 'แชนแนล': 1,
 'ยุค': 1,
 'เขา': 1,
 'มีชื่อเสียง': 1,
 'จาก': 1,
 'การ': 1,
 'คอลัมนิสต์': 1,
 'ผู้ดำเนินรายการ': 1,
 'สนทนา': 1,
 'เชิง': 1,
 'ข่าว': 1,
 'รายการ': 1,
 'ที่': 1,
 'สร้าง': 1,
 'ชื่อ': 1,
 'รายงาน': 1,
 'สงคราม': 1,
 'อ่าว': 1,
 'เปอร์เซีย': 1,
 'ช่วง': 1,
 'ต้นปี': 1,
 '2534': 1,
 'ทาง': 1,
 'ทีวีสี': 1,
 'ช่อง': 1,
 '3': 1,
 '9': 1,
 'อ.ส.ม.ท.': 1,
 'เครือ': 1,
 'เริ่ม': 1,
 'ผลิต': 1,
 'รายการโทรทัศน์': 1,
 'ครั้งแรก': 1,
 'นิวส์': 1,
 'ทอล์ก': 1,
 '2545': 1,
 'พ.ต.ท.': 1,
 'ทักษิณ': 1,
 'ชินวัตร': 1,
 'นายกรัฐมนตรี': 1,
 'ในขณะนั้น': 1,
 'มอบหมาย': 1,
 'ให้': 1,
 'สำนักงาน': 1,
 'คณะกรรมการ': 1,
 'ป้องกัน': 1,
 'ปราบปราม': 1,
 'ฟอกเงิน': 1,
 '(': 1,
 'ปปง.': 1,
 ')': 1,
 'เข้า': 1,
 'ตรวจสอบ': 1,
 'บัญชี': 1,
 'ธนาคาร': 1,
 'ชำระ': 1,
 'ภาษี': 1,
 'ย้อนหลัง': 1,
 'สื่อมวลชน': 1,
 'จำนวน': 1,
 'หนึ่ง': 1,
 'ซึ่ง': 1,
 'นั้น': 1,
 'มี': 1,
 'รวม': 1,
 'อยู่': 1,
 'ด้วย': 1,
 'has person': True,
 'has role': False}

In [ ]:
{'พล.ต.อ.': 1,
 'ชิด': 1,
 'ชัย': 1,
 ' ': 1,
 'เข้า': 1,
 'รับ': 1,
 'ตำแหน่ง': 1,
 'เมื่อ': 1,
 'วันที่': 1,
 '11': 1,
 'มีนาคม': 1,
 'พ.ศ.': 1,
 '2548': 1,
 'เป็น': 1,
 'รองนายกรัฐมนตรี': 1,
 'และ': 1,
 'รัฐมนตรี': 1,
 'ว่าการ': 1,
 'กระทรวงมหาดไทย': 1,
 'ต่อมา': 1,
 'ใน': 1,
 '5': 1,
 'เมษายน': 1,
 'ได้': 1,
 'รับมอบหมาย': 1,
 'ให้': 1,
 'ทำหน้าที่': 1,
 'รักษาการ': 1,
 'นายกรัฐมนตรี': 1,
 'รัฐบาล': 1,
 'พ.ต.ท.': 1,
 'ทักษิณ': 1,
 'ชินวัตร': 1,
 'ได้รับแต่งตั้ง': 1,
 'กระทรวงยุติธรรม': 1,
 '2': 1,
 'สิงหาคม': 1,
 'ปี': 1,
 '2550': 1,
 'ถูก': 1,
 'ตัด': 1,
 'สิทธิ': 1,
 'ทางการเมือง': 1,
 'เป็นเวลา': 1,
 'เนื่องจาก': 1,
 'กรรม': 1,
 'การบริหาร': 1,
 'พรรค': 1,
 'ไทย': 1,
 'รัก': 1,
 'ซึ่ง': 1,
 'ยุบ': 1,
 'คดี': 1,
 'พรรคการเมือง': 1,
 '2549': 1,
 'has person': False,
 'has role': False}

In [ ]:
{'เนวิน': 1,
 ' ': 1,
 'ชิด': 1,
 'ชอบ': 1,
 '()': 1,
 'ปัจจุบัน': 1,
 'เป็น': 1,
 'ประธาน': 1,
 'บริหาร': 1,
 'สโมสร': 1,
 'ฟุตบอล': 1,
 'บุรีรัมย์': 1,
 'ยูไนเต็ด': 1,
 'อดีต': 1,
 'เคย': 1,
 'สมาชิกสภาผู้แทนราษฎร': 1,
 'และ': 1,
 'รัฐมนตรี': 1,
 'หลาย': 1,
 'สมัย': 1,
 'เป็นหนึ่ง': 1,
 'ใน': 1,
 'บุคคล': 1,
 'ใกล้ชิด': 1,
 'คนสำคัญ': 1,
 'ของ': 1,
 'นายกรัฐมนตรี': 1,
 'ทักษิณ': 1,
 'ชินวัตร': 1,
 'สำคัญ': 1,
 'ที่': 1,
 'ทำให้': 1,
 'นาย': 1,
 'อภิสิทธิ์': 1,
 'เวช': 1,
 'ชา': 1,
 'ชีวะ': 1,
 'ได้': 1,
 'ดำรงตำแหน่ง': 1,
 'ปี': 1,
 'พ.ศ.': 1,
 '2551': 1,
 'has person': False,
 'has role': False}

In [ ]:
test_sent_features

0     {'เดือน': 1, 'ธันวาคม': 1, ' ': 1, 'ค.ศ.': 1, ...
1     {'แต่': 1, 'ต่อมา': 1, ' ': 1, 'คู่แข่ง': 1, '...
2     {'ภาย': 1, 'หลังจาก': 1, '[[': 1, 'การ': 1, 'โ...
3     {'[[': 1, 'ไฟล์': 1, ':': 1, 'President': 1, '...
4     {'วันที่': 1, ' ': 1, '29': 1, 'มกราคม': 1, 'ค...
5     {'วันที่': 1, ' ': 1, '30': 1, 'กันยายน': 1, '...
6     {'[[': 1, 'ไฟล์': 1, ':': 1, 'Obama': 1, ' ': ...
7     {'วันที่': 1, ' ': 1, '14': 1, 'กรกฎาคม': 1, '...
8     {'โอบามา': 1, 'ได้': 1, 'แจ้ง': 1, 'ว่า': 1, '...
9     {'[[': 1, 'ไฟล์': 1, ':': 1, 'ObamaSouthCaroli...
10    {'ใน': 1, 'เดือน': 1, 'ธันวาคม': 1, ' ': 1, 'ค...
11    {'[[': 1, 'ไฟล์': 1, ':': 1, 'BarackObama-Bask...
12    {'โอบามา': 1, 'ชนะ': 1, '[[': 1, 'รางวัล': 1, ...
13    {'สุภาพสตรี': 1, 'หมายเลข': 1, 'หนึ่ง': 1, 'คน...
14    {'สุภาพสตรี': 1, 'หมายเลข': 1, 'หนึ่ง': 1, 'คน...
15    {'ใน': 1, 'เดือน': 1, 'มกราคม': 1, ' ': 1, '20...
16    {'ระหว่าง': 1, 'นี้': 1, ' ': 1, 'โอบามา': 1, ...
17    {'นอกจากนี้': 1, ' ': 1, 'ยังมี': 1, 'เหล่

In [ ]:
test_sent_features[26]

{'พลเอก': 1,
 ' ': 1,
 'สนธิ': 1,
 'บุญ': 1,
 'ย': 1,
 'รัต': 1,
 'ก': 1,
 'ลิ': 1,
 'น': 1,
 'หัวหน้า': 1,
 'คณะ': 1,
 'ปฏิรูป': 1,
 'การปกครอง': 1,
 'ใน': 1,
 'ระบอบประชาธิปไตย': 1,
 'อัน': 1,
 'มี': 1,
 'พระมหากษัตริย์': 1,
 'ทรง': 1,
 'เป็น': 1,
 'ประมุข': 1,
 'ได้': 1,
 'นำ': 1,
 'ความ': 1,
 'กราบบังคมทูลพระกรุณา': 1,
 'ทราบฝ่าละอองธุลีพระบาท': 1,
 'ว่า': 1,
 'การบริหาร': 1,
 'ราชการ': 1,
 'แผ่นดิน': 1,
 'ของ': 1,
 'รัฐบาล': 1,
 'พัน': 1,
 'ตำรวจ': 1,
 'โท': 1,
 'ทักษิณ': 1,
 'ชินวัตร': 1,
 'นายกรัฐมนตรี': 1,
 'ก่อให้เกิด': 1,
 'ปัญหา': 1,
 'ความขัดแย้ง': 1,
 'แบ่ง': 1,
 'ฝ่าย': 1,
 'สลาย': 1,
 'ความรู้สึก': 1,
 'รู้': 1,
 'รัก': 1,
 'สามัคคี': 1,
 'คนใน': 1,
 'ชาติ': 1,
 'อย่าง': 1,
 'ที่': 1,
 'ไม่': 1,
 'เคย': 1,
 'ปรากฏ': 1,
 'มา': 1,
 'ก่อน': 1,
 'ประวัติศาสตร์': 1,
 'ไทย': 1,
 'ประชาชน': 1,
 'ส่วนหนึ่ง': 1,
 'เคลือบแคลง': 1,
 'สงสัย': 1,
 'ส่อ': 1,
 'ไป': 1,
 'ทาง': 1,
 'ทุจริต': 1,
 'ประพฤติมิชอบ': 1,
 'กว้างขวาง': 1,
 'หน่วยงาน': 1,
 'อิสระ': 1,
 'ถูก': 1,
 'การเมือง': 1,
